# How to configure multiple streaming modes at the same time

This guide covers how to configure multiple streaming modes at the same time.

## Setup

First we need to install the packages required

```bash
npm install @langchain/langgraph @langchain/openai
```

Next, we need to set API keys for OpenAI (the LLM we will use)

```bash
export OPENAI_API_KEY=your-api-key
```

Optionally, we can set API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_CALLBACKS_BACKGROUND="true"
export LANGCHAIN_API_KEY=your-api-key
```

## Define the graph

We'll be using a simple ReAct agent for this guide.

In [3]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from '@langchain/core/tools';
import { z } from 'zod';
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const model = new ChatOpenAI({
    model: "gpt-4o",
  });

const getWeather = tool((input) => {
    if (['sf', 'san francisco', 'san francisco, ca'].includes(input.location.toLowerCase())) {
        return 'It\'s 60 degrees and foggy.';
    } else {
        return 'It\'s 90 degrees and sunny.';
    }
}, {
    name: 'get_weather',
    description: 'Call to get the current weather.',
    schema: z.object({
        location: z.string().describe("Location to get the weather for."),
    })
})

const graph = createReactAgent({ llm: model, tools: [getWeather] });

## Stream Multiple

The stream mode

In [4]:
let inputs = { messages: [["user", "what's the weather in sf"]] };

for await (
  const chunk of await graph.stream(inputs, {
    streamMode: ["updates","debug"],
  })
) {
  for (const [node, values] of Object.entries(chunk)) {
    console.log(`Receiving update from node: ${node}`);
    console.log(values);
    console.log("\n====\n");
  }
}

Receiving update from node: 0
debug

====

Receiving update from node: 1
{
  type: 'task',
  timestamp: '2024-08-30T03:54:49.078Z',
  step: 1,
  payload: {
    id: '768110dd-6004-59f3-8671-6ca699cccd71',
    name: 'agent',
    input: { messages: [Array] },
    triggers: [ 'start:agent' ]
  }
}

====

Receiving update from node: 0
updates

====

Receiving update from node: 1
{
  agent: {
    messages: [
      AIMessage {
        "id": "chatcmpl-A1n0jsdqbFG937Om1bFGOePuWJkL2",
        "content": "",
        "additional_kwargs": {
          "tool_calls": [
            {
              "id": "call_unXrrn2pgYHPCq2ll7tTw9hb",
              "type": "function",
              "function": "[Object]"
            }
          ]
        },
        "response_metadata": {
          "tokenUsage": {
            "completionTokens": 17,
            "promptTokens": 58,
            "totalTokens": 75
          },
          "finish_reason": "tool_calls",
          "system_fingerprint": "fp_157b3831f5"
        